In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 1. No Cache

In [ ]:
import os
import time
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''

openai_client = OpenAI()

def response_text(openai_resp):
  return openai_resp.choices[0].message.content

model = 'gpt-3.5-turbo'
question = '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?'
message = [{'role': 'user', 'content': question }]

for _ in range(2):
  start_time = time.time()
  response = openai_client.chat.completions.create(model=model, messages=message)
  response = response_text(response)
  running_time = time.time() - start_time

  print(f'질문: {question}')
  print(f'소요 시간: {running_time:.2f}s')
  print(f'답변: {response}\n')

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 3.35s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을부터 겨울까지입니다. 이 기간 동안 한반도에는 추위와 바람이 강해지며, 일부 지역에서는 눈이 내리기도 합니다.해당 기단은 주로 10월부터 3월 사이에 영향을 미치며, 특히 11월부터 2월 사이에 가장 강하게 느껴집니다. 해가 바뀌면서 온도도 급격히 떨어지고 다양한 기후변화가 있을 수 있습니다. 이 기간 동안 적절한 준비와 방법으로 추위를 대비할 필요가 있습니다.

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 1.43s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을에서 겨울 사이인 10월부터 다음 해 3월까지입니다. 이 기간 동안 기단이 국내에 머무르면서 한겨울 강한 추위를 가져오게 됩니다.



## 2. cache

In [ ]:
class OpenAICache:
  def __init__(self, openai_client):
    self.openai_client = openai_client
    self.cache = {}

  def generate(self, prompt):
    if prompt not in self.cache:
      message = [{'role': 'user', 'content': prompt }]
      response = self.openai_client.chat.completions.create(model=model, messages=message)
      self.cache[prompt] = response_text(response)

    return self.cache[prompt]


openai_cache = OpenAICache(openai_client)

for _ in range(2):
  start_time = time.time()
  response = openai_cache.generate(question)
  running_time = time.time() - start_time

  print(f'질문: {question}')
  print(f'소요 시간: {running_time:.2f}s')
  print(f'답변: {response}\n')

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 1.64s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 보통 봄과 가을에 해당됩니다. 봄에는 4월부터 5월, 가을에는 9월부터 10월까지 주로 국내에 머무르게 됩니다. 이 기간 동안 대부분의 날씨는 서늘하고 건조하며 바람이 강하게 불 수도 있습니다.

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 0.00s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 보통 봄과 가을에 해당됩니다. 봄에는 4월부터 5월, 가을에는 9월부터 10월까지 주로 국내에 머무르게 됩니다. 이 기간 동안 대부분의 날씨는 서늘하고 건조하며 바람이 강하게 불 수도 있습니다.



## 3. similar search

In [ ]:
%%capture
! pip install chromadb

In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
class OpenAICache:
  def __init__(self, openai_client, semantic_cache):
    self.openai_client = openai_client
    self.cache = {}
    self.semantic_cache = semantic_cache

  def generate(self, prompt):
    if prompt not in self.cache:
      similar_doc = self.semantic_cache.query(query_texts=[prompt], n_results=1)
      if len(similar_doc['distances'][0]) > 0 and similar_doc['distances'][0][0] < 0.2:
        return similar_doc['metadatas'][0][0]['response']

      else:
        message = [{'role': 'user', 'content': prompt }]
        response = self.openai_client.chat.completions.create(model=model, messages=message)
        self.cache[prompt] = response_text(response)

        metadata = {'response':response_text(response)}
        self.semantic_cache.add(documents=[prompt], metadatas=[metadata], ids=[prompt])

    return self.cache[prompt]

In [ ]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_model = 'text-embedding-ada-002'
openai_ef = OpenAIEmbeddingFunction(api_key=os.environ['OPENAI_API_KEY'])

semantic_cache = chroma_client.create_collection(
    name='semantic_cache',
    embedding_function=openai_ef,
    metadata={'hnsw:space': 'cosine'}
)

openai_cache = OpenAICache(openai_client, semantic_cache)

questions = [
    '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
    '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
    '북태평양 기단과 오호츠크해 기단이 만나 한반도에 머무르는 기간은?',
    '국내에 북태평양 기단과 오호츠크해 기단이 함께 머무르는 기간은?'
]

for question in questions:
  start_time = time.time()
  response = openai_cache.generate(question)
  running_time = time.time() - start_time

  print(f'질문: {question}')
  print(f'소요 시간: {running_time:.2f}s')
  print(f'답변: {response}\n')

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 3.42s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을부터 겨울에 이르는 기간이며, 보통 10월부터 3월 사이에 국내를 방문하게 됩니다. 이 기단이 만나면 한반도에는 한파가 찾아오거나 강한 바람과 폭설이 예상되므로 춥고 추운 겨울철을 준비해야 합니다.

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 0.00s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을부터 겨울에 이르는 기간이며, 보통 10월부터 3월 사이에 국내를 방문하게 됩니다. 이 기단이 만나면 한반도에는 한파가 찾아오거나 강한 바람과 폭설이 예상되므로 춥고 추운 겨울철을 준비해야 합니다.

질문: 북태평양 기단과 오호츠크해 기단이 만나 한반도에 머무르는 기간은?
소요 시간: 0.56s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을부터 겨울에 이르는 기간이며, 보통 10월부터 3월 사이에 국내를 방문하게 됩니다. 이 기단이 만나면 한반도에는 한파가 찾아오거나 강한 바람과 폭설이 예상되므로 춥고 추운 겨울철을 준비해야 합니다.

질문: 국내에 북태평양 기단과 오호츠크해 기단이 함께 머무르는 기간은?
소요 시간: 0.52s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을부터 겨울에 이르는 기간이며, 보통 10월부터 3월 사이에 국내를 방문하게 됩니다. 이 기단이 만나면 한반도에는 한파가 찾아오거나 강한 바람과 폭설이 예상되므로 춥고 추운 겨울철을 준비해야 합니다.

